In [82]:
import csv


class IsearSubset:

    def __init__(self,
                 labels,
                 values):
        self.labels = labels
        self.values = values


class IsearDataSet:

    def __init__(self,
                 data=IsearSubset([], []),
                 target=IsearSubset([], []),
                 text_data=[]):
        self.__data = data
        self.__target = target
        self.__text_data = text_data

    def get_data(self):
        return self.__data.values

    def get_target(self):
        return self.__target.values

    def get_data_label_at(self, i):
        return self.__data.labels[i]

    def get_target_label_at(self, i):
        return self.__target.labels[i]

    def get_freetext_content(self):
        return self.__text_data


class NoSuchFieldException:

    def __init__(self, field_name):
        self.message = "No such field in dataset : " + field_name

    def get_message(self):
        return self.message


class IsearLoader:

    def load_isear(self, s_isear_path):
        f_isear = open(s_isear_path, "r")
        '''
        The isear file extracted for the purpose of this initial
        loading is a pipe delimited csv-like file with headings
        '''

        isear_reader = csv.reader(f_isear,
                                  delimiter="|",
                                  quotechar='"')
        i = 0
        entry_attributes = []
        text_data = []
        entry_target = []
        for isear_row in isear_reader:
            if i == 0:
                i = i + 1
                continue
            result = self.__parse_entry(isear_row,
                                        i,
                                        text_data)
            entry_attributes.append(result["attributes"])
            entry_target.append(result["target"])
            i = i + 1
        attributes_subset = IsearSubset(self.attribute_list,
                                        entry_attributes)
        target_subset = IsearSubset(self.target_list,
                                    entry_target)
        return IsearDataSet(attributes_subset,
                            target_subset,
                            text_data)

    def __parse_entry(self,
                      isear_row,  # The row of the entry
                      index,  # row number
                      text_data):  # the text data
        i_col = 0
        l_attributes = []
        l_target = []
        # start parsing the columns
        for isear_col in isear_row:
            # we need to know to which field we are refering
            # handling the excess columns
            if i_col >= len(CONST_ISEAR_CODES):
                break

            s_cur_col = CONST_ISEAR_CODES[i_col]

            # for further test this will tell whether we are in the SIT column,
            # which is a text column
            b_is_sit = bool(s_cur_col == "SIT")
            if b_is_sit:
                if self.provide_text:
                    # should be clear enough
                    text_data.append(isear_col)
            else:
                # should be an int

                if s_cur_col in self.attribute_list:
                    i_isear_col = int(isear_col)
                    l_attributes.append(i_isear_col)

                if s_cur_col in self.target_list:
                    i_isear_col = int(isear_col)
                    l_target.append(i_isear_col)
            # next column
            i_col = i_col + 1
        # we will return a pretty "free form" object
        return {"attributes": l_attributes,
                "target": l_target}

    def __init__(self,
                 attribute_list=[],
                 target_list=[],
                 provide_text=True):
        # list of attributes to extract, please refer to enums.py
        self.attribute_list = []
        self.set_attribute_list(attribute_list)
        # list of targets to extract
        self.target_list = []
        self.set_target_list(target_list)
        # provide the text, true by default
        self.provide_text = provide_text

    # compares attribute existence in the Isear labels
    def __check_attr_exists(self, attribute):
        return attribute in CONST_ISEAR_CODES

    def set_attribute_list(self, attrs):
        """Set a list of attributes to extract
        Args:
        attrs (list):  a list of strings refering Isear fields .
        Returns:
        self. in order to ease fluent programming (loader.set().set())
        Raises:
        NoSuchFieldException
        """
        self.attribute_list = []
        for attr in attrs:
            self.add_attribute(attr)
        return self

    def set_target_list(self, target):
        """Set a list of fields to extract as target
        Args:
        attrs (list):  a list of strings refering Isear fields .
        Returns:
        self. in order to ease fluent programming (loader.set().set())
        Raises:
        NoSuchFieldException
        """
        self.target_list = []
        for tgt in target:
            self.add_target(tgt)
        return self

    def set_provide_text(self, is_provide_text):
        """ Tell the extractor whether to load the free text field.
        Behaviour is true by default
        Args:
        is_provide_text (bool): whether to provide the text field or not
        Return
        self. For fluent API
        """
        self.provide_text = is_provide_text
        return self

    def add_attribute(self, attr):
        b_att_ex = self.__check_attr_exists(attr)
        if b_att_ex is not True:
            ex = NoSuchFieldException(attr)
            raise ex
        self.attribute_list.append(attr)
        return self

    def add_target(self, attr):
        b_att_ex = self.__check_attr_exists(attr)
        if b_att_ex is not True:
            ex = NoSuchFieldException(attr)
            raise ex
        self.target_list.append(attr)
        return self




In [83]:
"""
Isear : Constants
This file will incorporate all needed constants in the first time
"""

"""
Here are the ISEAR codes explained
ID : identifier of the subject
CITY : city in which lives the subject
COUN : country of the subject
1 "SWEDEN" 
2 "NORWAY" 
3 "F.R.G." 
4 "FINLAND" 
5 "GREECE" 
6 "HONG KONG" 
7 "LEBANON" 
8 "AUSTRIA" 
9 "AUSTRALIA" 
10 "BRAZIL" 
11 "BOTSWANA" 
12 "BULGARIA" 
13 "FRANCE" 
14 "ITALY" 
15 "JAPAN" 
16 "NEW ZEALAND" 
17 "NETHERLANDS" 
18 "PORTUGAL" 
19 "SPAIN" 
20 "ZAMBIA" 
21 "ZIMBABWE" 
22 "USA" 
23 "POLAND" 
24 "NIGERIA" 
25 "ISRAEL" 
26 "INDIA" 
27 "MALAWI" 
28 "SWITZERLAND" 
29 "CHILE" 
30 "CHINA MAINLAND" 
31 "YUGOSLAVIA" 
32 "COSTA RICA" 
33 "HONDURAS" 
34 "MEXICO" 
35 GUATEMALA" 
36 "VENEZUELA" 
37 "EL SALVADOR"
SUBJ :
SEX : the sex of the subject
1 MALE
2 FEMALE
AGE : the age of the subject
RELI : the religion of the subject
1 "PROTESTANT" 
2 "CATHOLIC" 
3 "JEWISH" 
4 "HINDU" 
5 "BUDDHIST" 
6 "NATIVE" 
7 "OTHERS" 
8 "ARELIGIOUS"
PRAC : is subject practising religion
1 TRUE
2 FALSE
FOCC & MOCC : Father and Mother occupation
1 "HOUSEWIFE" 2 "UNEMPLOYED" 3 "STUDENT" 4 "BLUE COLLAR UNTRAINED" 5 "BLUE COLLAR TRAINED" 6 "WHITE COLLAR NONACADEMIC" 7 "WHITE COLLAR ACADEMIC" 8 "SELF-EMPLOYED NONACADEMIC" 9 "SELF-EMPLOYED ACADEMIC"
FIEL : subject's field of study
1 "PSYCHOLOGY" 2 "SOCIAL SCIENCES" 3 "LANGUAGES" 4 "FINE ARTS" 5 "LAW" 6 "NATURAL SCIENCE" 7 "ENGINEERING" 8 "MEDICAL" 9 "OTHER"
EMOT : subject's main emotion
1 JOY
2 FEAR
3 ANGER 
4 SADNESS
5 DISGUST
6 SHAME
7 GUILT
SIT : free text description (core of training)
-------------------- 
Event Quantification
--------------------
WHEN : when did that happen
1 DAYS AGO 2 WEEKS AGO 3 MONTHS AGO 4 YEARS AGO
LONG : how long did it last
1 MINUTES 2 HOUR 3 HOURS 4 DAY OR MORE
INTS : intensity score from 1 to 4
--------------------
Reaction
--------------------
ERGO : Ergotropic arousal (breathing, heart beat, etc) from 0 to 4
TROPHO : Trophophic arousal (lump, stomach, crying...) from 0 to 3
TEMPER : Felt temperature (-1 to 2)
--------------------
Expressive behaviour
--------------------
MOVE : from -1 (withdrawing) to +1 (forward)
EXPRES : nonverbal activity, from 0 to 6
PARAL : speech melody and tempo change from 0 to 3
EXP1 : if laughing/smilig 1, else 0
EXP2 : if sobbing/Crying
EXP10 : if moving against
CON : subject efforts to control
1 NONE 2 A LITTLE 3 VERY MUCH 0 NA
EXPC : situation expected to occur
0 NA 1 NOT 2 A LITTLE 3 VERY MUCH
PLEA : situation is pleasant
FAIR : fairness of the situation
MORL : morality of the situation
SELF : consequence on self estime
RELA : consequence on relationship with persons involved
1 NEGATIVE 2 NONE 3 POSITIVE 0 NA
PLAN : situation hindered current plans
0 HELPED 2 NO MATTER 3 HINDERED 0 NA
CAUS : who caused the situation
O NA 1 SELF 2 CLOSE 3 OTHER 4 IMPERSONNAL
COPING : ability to cope
1 NO ACTION NECESSARY
2 MANAGEABLE
3 ESCAPABLE SITUATION
4 DENIAL
5 DOMINATED
 
"""

# ID : identifier of the subject
# CITY : city in which lives the subject

# COUN : country of the subject
COUN = ["SWEDEN", "NORWAY", "F.R.G.", "FINLAND",
        "GREECE", "HONG KONG", "LEBANON", "AUSTRIA",
        "AUSTRALIA", "BRAZIL", "BOTSWANA", "BULGARIA",
        "FRANCE", "ITALY", "JAPAN", "NEW ZEALAND",
        "NETHERLANDS", "PORTUGAL", "SPAIN",
        "ZAMBIA", "ZIMBABWE", "USA", "POLAND", "NIGERIA",
        "ISRAEL", "INDIA", "MALAWI", "SWITZERLAND", "CHILE",
        "CHINA MAINLAND", "YUGOSLAVIA", "COSTA RICA", "HONDURAS",
        "MEXICO", "GUATEMALA", "VENEZUELA", "EL SALVADOR"]

# SUBJ : Identifier of the subject

# SEX : the sex of the subject
SEX = ["MALE", "FEMALE"]

# AGE : the age of the subject

# RELI : the religion of the subject
RELI = ["PROTESTANT", "CATHOLIC", "JEWISH", "HINDU",
        "BUDDHIST", "NATIVE", "OTHERS", "ARELIGIOUS"]

# PRAC : is subject practising religion
PRAC = ["TRUE", "FALSE"]

# FOCC & MOCC : Father and Mother occupation
POCC = ["HOUSEWIFE", "UNEMPLOYED", "STUDENT",
        "BLUE COLLAR UNTRAINED", "BLUE COLLAR TRAINED",
        "WHITE COLLAR NONACADEMIC", "WHITE COLLAR ACADEMIC",
        "SELF-EMPLOYED NONACADEMIC", "SELF-EMPLOYED ACADEMIC"]

# FIEL : subject's field of study
FIEL = ["PSYCHOLOGY", "SOCIAL SCIENCES", "LANGUAGES", "FINE ARTS",
        "LAW", "NATURAL SCIENCE", "ENGINEERING", "MEDICAL", "OTHER"]

# EMOT : subject's main emotion
EMOT = ["JOY", "FEAR", "ANGER", "SADNESS", "DISGUST", "SHAME", "GUILT"]

# SIT : free text description (core of training)

# --------------------
# Event Quantification
# --------------------
# WHEN : when did that happen
WHEN = ["DAYS AGO", "WEEKS AGO", "MONTHS AGO", "YEARS AGO"]

# LONG : how long did it last
LONG = ["MINUTES", "HOUR", "HOURS", "DAY OR MORE"]

# INTS : intensity score from 1 to 4

# --------------------
# Reaction
# --------------------
# ERGO : Ergotropic arousal (breathing, heart beat, etc) from 0 to 4

# TROPHO : Trophophic arousal (lump, stomach, crying...) from 0 to 3

# TEMPER : Felt temperature (-1 to 2)

# --------------------
# Expressive behaviour
# --------------------
# MOVE : from -1 (withdrawing) to +1 (forward)

# EXPRES : nonverbal activity, from 0 to 6

# PARAL : speech melody and tempo change from 0 to 3

# EXP1 : if laughing/smilig 1, else 0
# EXP2 : if sobbing/Crying
# EXP10 : if moving against

# CON : subject efforts to control
CON = ["NA", "NONE", "A LITTLE", "VERY MUCH"]

# EXPC : situation expected to occur
EXPC = CON
# PLEA : situation is pleasant
PLEA = CON
# FAIR : fairness of the situation
FAIR = CON
# MORL : morality of the situation
MORL = CON
# SELF : consequence on self estime
SELF = CON

# RELA : consequence on relationship with persons involved
RELA = ["NA", "NEGATIVE", "NONE", "POSITIVE"]

# PLAN : situation hindered current plans
PLAN = ["NA", "HELPED", "NO_MATTER", "HINDERED"]

# CAUS : who caused the situation
CAUS = ["NA", "SELF", "CLOSE", "OTHER", "IMPERSONNAL"]

# COPING : ability to cope
COPING = ["NO ACTION NECESSARY", "MANAGEABLE",
          "ESCAPABLE SITUATION", "DENIAL", "DOMINATED"]

# ---------------------------------------
# LIST OF AVAILABLE FIELDS/ATTRIBUTES
# ---------------------------------------
CONST_ISEAR_CODES = ["ID", "CITY", "COUN", "SUBJ", "SEX",
                     "AGE", "RELI", "PRAC", "FOCC", "MOCC",
                     "FIEL", "EMOT", "WHEN", "LONG", "INTS",
                     "ERGO", "TROPHO", "TEMPER", "EXPRES",
                     "MOVE", "EXP1", "EXP2", "EXP10", "PARAL",
                     "CON", "EXPC", "PLEA", "PLAN", "FAIR",
                     "CAUS", "COPING", "MORL", "SELF", "RELA",
                     "VERBAL", "NEUTRO", "Field1", "Field3",
                     "Field2", "MYKEY", "SIT", "STATE"]

In [84]:
attributes = ['EMOT']
target = ['TROPHO','TEMPER']
loader = IsearLoader(attributes, target, True)
data = loader.load_isear('isear.csv')

In [85]:
target = data.get_data()


In [86]:
print (target)


[[1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [2], [3], [4], [5], [6], [7], [1], [3], [4],

In [87]:
words = data.get_freetext_content()

In [88]:
print (len(words))

7666


In [89]:
import numpy as np
dataset = [[],[]]
print (dataset)

[[], []]


In [90]:
for i in range(7666):
    dataset[0].append(words[i])
    

In [91]:
print (dataset)

[['During the period of falling in love, each time that we met and á especially when we had not met for a long time.', 'When I was involved in a traffic accident.', 'When I was driving home after  several days of hard work, there á was a motorist ahead of me who was driving at 50 km/hour and á refused, despite his low speeed to let me overtake.', 'When I lost the person who meant the most to me. ', "The time I knocked a deer down - the sight of the animal's á injuries and helplessness.  The realization that the animal was á so badly hurt that it had to be put down, and when the animal á screamed at the moment of death.", 'When I did not speak the truth.', 'When I caused problems for somebody because he could not keep the á appointed time and this led to various consequences.', 'When I got a letter offering me the Summer job that I had applied á for.  ', 'When I was going home alone one night in Paris and a man came up á behind me and asked me if I was not afraid to be out alone so á la

In [92]:
for i in range(7666):
    dataset[1].append(target[i][0])

In [93]:
print (dataset)

[['During the period of falling in love, each time that we met and á especially when we had not met for a long time.', 'When I was involved in a traffic accident.', 'When I was driving home after  several days of hard work, there á was a motorist ahead of me who was driving at 50 km/hour and á refused, despite his low speeed to let me overtake.', 'When I lost the person who meant the most to me. ', "The time I knocked a deer down - the sight of the animal's á injuries and helplessness.  The realization that the animal was á so badly hurt that it had to be put down, and when the animal á screamed at the moment of death.", 'When I did not speak the truth.', 'When I caused problems for somebody because he could not keep the á appointed time and this led to various consequences.', 'When I got a letter offering me the Summer job that I had applied á for.  ', 'When I was going home alone one night in Paris and a man came up á behind me and asked me if I was not afraid to be out alone so á la

In [94]:
import pandas as pd
dataset = np.array(dataset)

In [95]:
print (dataset)

[['During the period of falling in love, each time that we met and á especially when we had not met for a long time.'
  'When I was involved in a traffic accident.'
  'When I was driving home after  several days of hard work, there á was a motorist ahead of me who was driving at 50 km/hour and á refused, despite his low speeed to let me overtake.'
  ...
  'I was at home and I heard a loud sound of spitting outside the á door.  I thought that one of my family members would step on the spit á and bring the germs in the house.'
  'I did not do the homework that the teacher had asked us to do.  I á was scolded immediately.'
  'I had shouted at my younger brother and he was always afraid when á I called out loudly.']
 ['1' '2' '3' ... '5' '6' '7']]


In [96]:
X_train = dataset[0]

In [97]:
print (X_train)
print (len(X_train))

['During the period of falling in love, each time that we met and á especially when we had not met for a long time.'
 'When I was involved in a traffic accident.'
 'When I was driving home after  several days of hard work, there á was a motorist ahead of me who was driving at 50 km/hour and á refused, despite his low speeed to let me overtake.'
 ...
 'I was at home and I heard a loud sound of spitting outside the á door.  I thought that one of my family members would step on the spit á and bring the germs in the house.'
 'I did not do the homework that the teacher had asked us to do.  I á was scolded immediately.'
 'I had shouted at my younger brother and he was always afraid when á I called out loudly.']
7666


In [98]:
y = dataset[1]
print (len(y))

7666


In [99]:
y123 = []
X_train123 = []
for i in range(len(y)):
    if y[i] != '5':
        if y[i] != '6':
           if y[i] != '7': 
    #           print (y[i])
               y123.append(y[i])
               X_train123.append(X_train[i])

y = y123
X_train = X_train123
ytemp = y
print (len(y))
print (len(X_train))

4381
4381


In [100]:
print (y)

['1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1', '2', '3', '4', '1',

In [101]:

#1 JOY
#2 FEAR
#3 ANGER 
#4 SADNESS
#5 DISGUST
#6 SHAME
#7 GUILT

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 4381):
    review = re.sub('[^a-zA-Z]', ' ', X_train[i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)
    


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tejasdeoras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [102]:
print (corpus)

['period fall love time met especi met long time', 'involv traffic accid', 'drive home sever day hard work motorist ahead drive km hour refus despit low speeed let overtak', 'lost person meant', 'got letter offer summer job appli', 'go home alon one night pari man came behind ask afraid alon late night', 'talk parti first time long friend came interrupt us left', 'friend ask go new year parti', 'day feel close partner friend feel peac also experi close contact peopl regard greatli', 'everi time imagin someon love could contact seriou ill even death', 'obvious unjustli treat possibl elucid', 'think short time live relat period life think use short time', 'girlfriend taken exam went parent place', 'first time realiz mean death', 'car overtak anoth forc drive road', 'recent thought hard work take studi one want tri someth els read theoret book english understand', 'pass examin think well', 'one arrang meet someon person arriv late meantim one start think could gone wrong e g traffic accid

In [103]:
#corpus1 = []
#for elem in corpus:
#    b = TextBlob(elem)
#    corpus1.append(b.correct())
    

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train = cv.fit_transform(corpus).toarray()
print (X_train.shape)
#max_features = 1500

(4381, 4557)


In [104]:
print (X_train.shape)
print(X_train)
y = np.asarray(y)
y= y.astype('int')

(4381, 4557)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [105]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.2, random_state=0)
print (corpus)
from sklearn import svm
classifier = svm.SVC(gamma='scale', decision_function_shape='ovo')
classifier.fit(X_train, y_train)

['period fall love time met especi met long time', 'involv traffic accid', 'drive home sever day hard work motorist ahead drive km hour refus despit low speeed let overtak', 'lost person meant', 'got letter offer summer job appli', 'go home alon one night pari man came behind ask afraid alon late night', 'talk parti first time long friend came interrupt us left', 'friend ask go new year parti', 'day feel close partner friend feel peac also experi close contact peopl regard greatli', 'everi time imagin someon love could contact seriou ill even death', 'obvious unjustli treat possibl elucid', 'think short time live relat period life think use short time', 'girlfriend taken exam went parent place', 'first time realiz mean death', 'car overtak anoth forc drive road', 'recent thought hard work take studi one want tri someth els read theoret book english understand', 'pass examin think well', 'one arrang meet someon person arriv late meantim one start think could gone wrong e g traffic accid

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovo', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [106]:
#from sklearn.model_selection import cross_val_score
#accuracy = cross_val_score(estimator = classifier, X = X_train, y = y, cv = 10, n_jobs = -1, scoring="accuracy")

In [124]:
yhat = classifier.predict(X_test)
print (len(y_test))

877


In [108]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, yhat)
print (acc)

0.6909920182440137


In [109]:
#Please specify your own path

import urllib
import speech_recognition as sr
import subprocess
import os
import json

#url = 'http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/VolkswagenGTIReview.mp4'
#mp4file = urllib.request.urlopen(url)

#Changes made here
#with open("/Users/tejasdeoras/Documents/test.mp4", "wb") as handle:
#    handle.write(mp4file.read())

import subprocess

#command = "ffmpeg -i /Users/tejasdeoras/Documents/test.mp4 -ab 160k -ac 2 -ar 44100 -vn test.wav"

#subprocess.call(command, shell=True)

command = "ffmpeg -i /Users/tejasdeoras/Documents/test.wav -f segment -segment_time 10 -c copy /Users/tejasdeoras/Documents/Aud/out%03d.wav"

subprocess.call(command, shell=True)


directory = '/Users/tejasdeoras/Documents/Aud/'

In [110]:
import re
file_order = []
trans = []
for filename in os.listdir(directory):
    if filename != '.DS_Store':
        print (filename)
        path = directory + filename
        start = 'out'
        end = '.wav'
        index = filename[filename.find(start)+len(start):filename.rfind(end)]
        print (index)
        index = int(index)
        print (index)
        command = 'curl -X POST -u ' + '"apikey:-ym-g_cKFSARHtqNBaWEBg8tgZqSJiC2QIKpV0mFln8p" ' +  '--header ' + '"Content-Type: audio/wav"' +  ' --data-binary @' + path + ' "https://stream.watsonplatform.net/speech-to-text/api/v1/recognize"'
        temp = command
        print (command)
        output = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
        jsonS,_ = output.communicate()
        d1 = json.loads(jsonS)
        print (d1)
        trans.append([index,d1])
        file_order.append(index)

out009.wav
009
9
curl -X POST -u "apikey:-ym-g_cKFSARHtqNBaWEBg8tgZqSJiC2QIKpV0mFln8p" --header "Content-Type: audio/wav" --data-binary @/Users/tejasdeoras/Documents/Aud/out009.wav "https://stream.watsonplatform.net/speech-to-text/api/v1/recognize"
{'results': [{'alternatives': [{'confidence': 0.74, 'transcript': 'get shot you think out of me now '}], 'final': True}, {'alternatives': [{'confidence': 0.65, 'transcript': 'I am the one who knocks '}], 'final': True}], 'result_index': 0}
out008.wav
008
8
curl -X POST -u "apikey:-ym-g_cKFSARHtqNBaWEBg8tgZqSJiC2QIKpV0mFln8p" --header "Content-Type: audio/wav" --data-binary @/Users/tejasdeoras/Documents/Aud/out008.wav "https://stream.watsonplatform.net/speech-to-text/api/v1/recognize"
{'results': [{'alternatives': [{'confidence': 0.96, 'transcript': 'you clearly '}], 'final': True}, {'alternatives': [{'confidence': 0.88, 'transcript': "don't know who you're talking to so let me clue you in I am not in danger Skyler I am the danger a guy opens

In [111]:
ftrans = []
for i in range(len(trans)):
    if "results" in trans[i][1]:
            for j in range(len(trans)):
                if trans[j][0] == i:
                    print (trans[j][0])
                    if not 'results' in trans[j][1]:
                       ftrans.append("None")
                    else:
                       sen = ''
                       for m in range(len(trans[j][1]['results'])):
                           x = trans[j][1]["results"][m]["alternatives"][0]["transcript"]
                           print (x)
                           sen = sen + x
                       ftrans.append(sen)
           
    else:
        ftrans.append("None")




0
well 
I've said it before if you are in danger we go to the police I don't wanna hear about lease you not 
1
say that lightly 
I know what he could do to this family but if it's the only real choice we have if it's either that or you can shop when the 
2
working your way here 
some hardened criminal Walt you work in over your head that's what we tell them and that's the truth was not the truth of 
3
course it is 
the school teacher cancer desperate for money we're done roped into working unable to even quit you told me that your cell 
4
fault 
S. king 
please let's both of us stop trying to justify this whole thing I did meet your in 
5
danger 
who were you talking to right now 
who is it you think you see 
6
do you know how much I make here I mean even if I told you you wouldn't believe it 
do you know what would happen if I certainly 
7
cited to stop going into work the business big enough that it could be listed on the nasdaq goes belly up 
disappears it ceases to exist without me

In [112]:
print (ftrans)
print (len(ftrans))


["well I've said it before if you are in danger we go to the police I don't wanna hear about lease you not ", "say that lightly I know what he could do to this family but if it's the only real choice we have if it's either that or you can shop when the ", "working your way here some hardened criminal Walt you work in over your head that's what we tell them and that's the truth was not the truth of ", "course it is the school teacher cancer desperate for money we're done roped into working unable to even quit you told me that your cell ", "fault S. king please let's both of us stop trying to justify this whole thing I did meet your in ", 'danger who were you talking to right now who is it you think you see ', "do you know how much I make here I mean even if I told you you wouldn't believe it do you know what would happen if I certainly ", 'cited to stop going into work the business big enough that it could be listed on the nasdaq goes belly up disappears it ceases to exist without me ',

In [113]:
#print (corpus)
import re
import nltk
noindex = []
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus1 = []
for i in range(0, len(ftrans)):
    if ftrans[i] != 'None':
            review = re.sub('[^a-zA-Z]', ' ', ftrans[i])
            review = review.lower()
            review = review.split()
            ps = PorterStemmer()
            review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
            review = ' '.join(review)
            corpus1.append(review)
    else:
            noindex.append(i)

print (len(noindex))
print (len(corpus))
print (len(corpus1))
corpus2 = corpus + corpus1
#print (corpus2)


from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
dataset1 = cv.fit_transform(corpus2).toarray()
#print (dataset)


#print (len(X_test[len(corpus)-1:-1]))
#print (X_test[len(corpus)-1:-1])

X_train = dataset1[:len(corpus),:]
X_test = dataset1[len(corpus)-1:-1,:]
print (len(X_train))
print (len(X_test))

from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y)



predictions = classifier.predict(X_test)

print (predictions)


#1 JOY
#2 FEAR
#3 ANGER 
#4 SADNESS
#5 DISGUST
#6 SHAME
#7 GUILT

emotion = []
counter = 0
for i in range(0,len(ftrans)):
    if i in noindex:
       emotion.append('None')
    else:
       if predictions[counter] == 1:
          emotion.append('JOY')
       elif predictions[counter] == 2:
          emotion.append('FEAR')
       elif predictions[counter] == 3:
          emotion.append('ANGER')
       elif predictions[counter] == 4:
          emotion.append('SADNESS')
       elif predictions[counter] == 5:
          emotion.append('SHAME')
       elif predictions[counter] == 6:
          emotion.append('GUILT')
       else:
          emotion.append('NEG')
       counter = counter + 1

print (emotion)
        

        



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tejasdeoras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
0
4381
12
4381
12
[4 1 1 3 2 3 1 4 1 4 4 1]
['SADNESS', 'JOY', 'JOY', 'ANGER', 'FEAR', 'ANGER', 'JOY', 'SADNESS', 'JOY', 'SADNESS', 'SADNESS', 'JOY']


In [114]:
import librosa as lr
import numpy as np
import pandas as pd
import os

# Extraction of spectral features 
def spectral_features(y,sr,stft):
    # 1. MFCC (40 most important features)
    mfccs = lr.feature.mfcc(y=y, sr=sr, n_mfcc=40)
    mean_mfccs = np.mean(mfccs.T,axis=0) 
    # 2. Pitch Chroma
    p_chroma = lr.feature.chroma_stft(S=stft, sr=sr)
    mean_p_chroma = np.mean(p_chroma.T,axis=0)
    # 3. Spectral Centroid
    sp_centroid = lr.feature.spectral_centroid(y=y, sr=sr)
    mean_sp_centroid = np.mean(sp_centroid.T,axis=0)
    # 4. Spectral Skewness/Contrast
    contrast = lr.feature.spectral_contrast(S=stft, sr=sr)
    mean_contrast = np.mean(contrast.T,axis=0)
    
    return mean_mfccs,mean_p_chroma,mean_sp_centroid,mean_contrast

# Extraction of prosodic features
def prosodic_features(y):
    # 1. Zero crossing rate
    zcr = lr.feature.zero_crossing_rate(y)
    mean_zcr = np.mean(zcr.T,axis=0)
    # 2. Root mean square and energy
    rmse = lr.feature.rmse(y=y)
    mean_rmse = np.mean(rmse.T,axis=0)
    
    return mean_zcr,mean_rmse

def qualitative_features(y,sr):
    # 1. For tone analysis, we calculate mel frequencies
    mel = lr.feature.melspectrogram(y, sr=sr)
    mean_mel = np.mean(mel.T,axis=0)
    
    return mean_mel

# Function to create data. We use above feature extractions to generate columns (features) and add them as rows in the dataframe
def create_data():
    
    data = []
    out = []
    
    path = '/Users/tejasdeoras/Documents/Aud/'
    list_dir = os.listdir(path)
    
    for file in list_dir:
    
        emo = file.split('_')
        if file.endswith(".wav"):
            #print(file)

            # Loading data file to librosa and calculate y and sample rate
            y, sr = lr.load(path+file)
            # Generate short term Fourier Transform (stft)
            stft = np.abs(lr.stft(y)) 

            f_mfccs,f_p_chroma,f_sp_centroid,f_contrast = spectral_features(y,sr,stft)
            f_zcr,f_rmse = prosodic_features(y)
            f_mel = qualitative_features(y,sr)

            row = np.concatenate((f_mfccs,f_p_chroma,f_sp_centroid,f_contrast,f_zcr,f_rmse,f_mel))

            data.append(row)
            out.append(emo)

    print(out)
    return pd.DataFrame(data)


In [115]:
# loading json and creating model
from keras.models import model_from_json
json_file = open('/Users/tejasdeoras/Downloads/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/Users/tejasdeoras/Downloads/Model_Emotion_Speech.h5")
print("Loaded model from disk")

Loaded model from disk


In [116]:

data_tmp = create_data()

to_drop = [169, 179, 180, 181, 182, 183, 187, 188, 189]

test = data_tmp.drop(data_tmp.columns[to_drop], axis=1)

test.columns = range(test.shape[1])

print(test.shape)

test_x =np.expand_dims(test.values, axis=2)
preds = loaded_model.predict(test_x,batch_size=1, verbose=1)

emotion_dict = {1:'happy',2:'sad',4:'fearful',3:'angry'}
preds1=preds.argmax(axis=1)
print(preds1)
#print(emotion_dict[preds1])
# evaluate loaded model on test data
#loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
#score = loaded_model.evaluate(test, y_test, verbose=0)
#print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

[['out009.wav'], ['out008.wav'], ['out000.wav'], ['out001.wav'], ['out003.wav'], ['out002.wav'], ['out006.wav'], ['out007.wav'], ['out011.wav'], ['out005.wav'], ['out004.wav'], ['out010.wav']]
(12, 181)
12/12 [==============================] - 0s 10ms/step
[0 3 3 3 3 3 0 0 1 2 3 2]


In [117]:
print (file_order)
result_temp = [1] * len(file_order)
print (result_temp)
for i in range(len(file_order)):
    result_temp[i] = preds1[file_order[i]]
print (len(result_temp))
preds1 = result_temp

[9, 8, 0, 1, 3, 2, 6, 7, 11, 5, 4, 10]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
12


In [118]:



emotion1 = []
counter = 0
for i in range(0,len(preds1)):
       if preds1[counter] == 0:
          emotion1.append('JOY')
       elif preds1[counter] == 1:
          emotion1.append('SADNESS')
       elif preds1[counter] == 2:
          emotion1.append('FEAR')
       elif preds1[counter] == 3:
          emotion1.append('ANGER')
       elif preds1[counter] == 5:
          emotion1.append('SHAME')
       elif preds1[counter] == 6:
          emotion1.append('GUILT')
       else:
          emotion1.append('NEG')
       counter = counter + 1

print (emotion1)

['FEAR', 'SADNESS', 'JOY', 'ANGER', 'ANGER', 'ANGER', 'JOY', 'JOY', 'FEAR', 'ANGER', 'ANGER', 'ANGER']


In [119]:
emotion_final = emotion + emotion1
from collections import Counter
priority = []
col_count = Counter(emotion_final)
print (col_count)
for letter, count in col_count.most_common(len(set(emotion_final))):
    priority.append(letter)
print (priority)

Counter({'JOY': 8, 'ANGER': 8, 'SADNESS': 5, 'FEAR': 3})
['JOY', 'ANGER', 'SADNESS', 'FEAR']


In [120]:
v_emot = []
for i in range(len(preds1)):
    if emotion1[i] == emotion[i]:
       v_emot.append(emotion1[i])
    elif emotion[i] == 'None':
       v_emot.append(emotion1[i]) 
    else:
       if priority.index(emotion1[i]) < priority.index(emotion[i]):
          v_emot.append(emotion1[i])
       else:
          v_emot.append(emotion[i])
print (v_emot)

['SADNESS', 'JOY', 'JOY', 'ANGER', 'ANGER', 'ANGER', 'JOY', 'JOY', 'JOY', 'ANGER', 'ANGER', 'JOY']


In [121]:
print (emotion)
print (emotion1)

['SADNESS', 'JOY', 'JOY', 'ANGER', 'FEAR', 'ANGER', 'JOY', 'SADNESS', 'JOY', 'SADNESS', 'SADNESS', 'JOY']
['FEAR', 'SADNESS', 'JOY', 'ANGER', 'ANGER', 'ANGER', 'JOY', 'JOY', 'FEAR', 'ANGER', 'ANGER', 'ANGER']


In [122]:
len_emot = len(emotion)
import datetime
time_ser = []
a = datetime.datetime(100,1,1,0,0,0)
time_ser.append(str(a.time()).replace("'",''))
for i in range(len_emot*5):
    a = a + datetime.timedelta(0,10)
    time_ser.append(str(a.time()).replace("'",''))
    time_ser.append(str(a.time()).replace("'",''))
print (time_ser)

['00:00:00', '00:00:10', '00:00:10', '00:00:20', '00:00:20', '00:00:30', '00:00:30', '00:00:40', '00:00:40', '00:00:50', '00:00:50', '00:01:00', '00:01:00', '00:01:10', '00:01:10', '00:01:20', '00:01:20', '00:01:30', '00:01:30', '00:01:40', '00:01:40', '00:01:50', '00:01:50', '00:02:00', '00:02:00', '00:02:10', '00:02:10', '00:02:20', '00:02:20', '00:02:30', '00:02:30', '00:02:40', '00:02:40', '00:02:50', '00:02:50', '00:03:00', '00:03:00', '00:03:10', '00:03:10', '00:03:20', '00:03:20', '00:03:30', '00:03:30', '00:03:40', '00:03:40', '00:03:50', '00:03:50', '00:04:00', '00:04:00', '00:04:10', '00:04:10', '00:04:20', '00:04:20', '00:04:30', '00:04:30', '00:04:40', '00:04:40', '00:04:50', '00:04:50', '00:05:00', '00:05:00', '00:05:10', '00:05:10', '00:05:20', '00:05:20', '00:05:30', '00:05:30', '00:05:40', '00:05:40', '00:05:50', '00:05:50', '00:06:00', '00:06:00', '00:06:10', '00:06:10', '00:06:20', '00:06:20', '00:06:30', '00:06:30', '00:06:40', '00:06:40', '00:06:50', '00:06:50', '00

In [123]:
len_emot = len(emotion)

import pysrt
file = pysrt.SubRipFile()
counter2 = 0
for i in range(len(emotion)):
        sub = pysrt.SubRipItem(i + 1, start= time_ser[counter2] + ',000', end= time_ser[counter2 + 1] + ',000', text= v_emot[i])
        counter2 = counter2 + 2
        file.append(sub)
file.save('/Users/tejasdeoras/Documents/my_movie.srt')